# Functions #



*Item 19 : Never Unpack more than three variables when functions return multiple values*

In [5]:
def get_avg_ratio(numbers) :
    average = sum(numbers)/len(numbers)
    scaled = [x/average for x in numbers]
    scaled.sort(reverse=True)
    return scaled

lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]
longest, *middle, shortest = get_avg_ratio(lengths)
k=5
print (f"Longest  : {longest:>4.0%} {k:%}")
print (f"Shortest : {shortest:>4.0%}")


## use a lightweight class or namedtuple if you need to return more than 3

Longest  : 108% 500.000000%
Shortest :  89%


*Item 20 : Prefer raising exceptions to returning None*

Nones can be confused with a 0 return (if nr is zero) . Easy programmer error is possible


In [6]:
## use exceptions and use type annotations .. lets try that

def careful_divide(a : float, b : float) -> float :
    try :
        return a/b
    except ZeroDivisionError as e :
        raise ValueError("Invalid inputs")

careful_divide(5, 6)
try :
    careful_divide(8, 0)
except ValueError as e :
    print (e)
careful_divide(0, 8)

careful_divide(9, 'a')
    

Invalid inputs


TypeError: unsupported operand type(s) for /: 'int' and 'str'

*Item 21 : Know How Closures interact with variable scope*

In [7]:
def sort_priority(values, group) :
    found = False
    def helper(x) :
        if x in group :
            ## if found is not declared nonlocal, the assignment will create a new local variable 
            ## and shit wont work
            nonlocal found
            found = True
            return (0, x)
        return (1, x)
    
    values.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

f = sort_priority(numbers, group)
print (numbers, f)

[2, 3, 5, 7, 1, 4, 6, 8] True


In [8]:
## Better way

class Sorter :
    def __init__(self, group) :
        self.group = group
        self.found = False
    def __call__(self, x) :
        if x in self.group :
            self.found = True
            return (0, x)
        return (1, x)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

s = Sorter(group)
numbers.sort(key=s)
print (numbers)
print (s.found)

[2, 3, 5, 7, 1, 4, 6, 8]
True


*Item 22 : Reduce visual noise with variable positional arguments*


In [9]:
def log(message, *values) :
    if not values :
        print (message)
    else :
        print ("Values is ", values)
        values_str = ",".join(str(x) for x in values)
        print (f"{message}: {values_str}")
        
log("hello")
log("hello", [1, 2, 3])
log ("hello", 1, 2, 3)

hello
Values is  ([1, 2, 3],)
hello: [1, 2, 3]
Values is  (1, 2, 3)
hello: 1,2,3


In [10]:
favs = [1, 2, 3]
log("Foo", *favs)

Values is  (1, 2, 3)
Foo: 1,2,3


In [11]:
log("foo", favs)

Values is  ([1, 2, 3],)
foo: [1, 2, 3]


*Item 23: Provide Optional Behavior with keyword arguments*

In [12]:
def remainder(number, divisor) : return number % divisor

d = {'divisor' : 7, 'number' : 20}
print (remainder(**d))

6


In [13]:
def print_params(**kwargs) :
    for k, v in kwargs.items() :
        print (f"{k} -> {v}")

print_params(**d)

divisor -> 7
number -> 20


In [14]:
print_params(name="foo", age=29)

name -> foo
age -> 29


*Item 24 : Use None and Docstrings to Specify Dynamic Default Arguments*

In [15]:
## Broken
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()) :
    print (f"{when}: {message}")
    
log("Hello")
sleep(1)
log("Dude")

2021-03-30 20:52:28.172127: Hello
2021-03-30 20:52:28.172127: Dude


In [16]:
## Correct

from time import sleep
from datetime import datetime

def log(message, when=None) :
    if not when :
        when = datetime.now()
    print (f"{when} : {message}")
    
log("Hello")
sleep(1)
log("Dude")

2021-03-30 20:54:18.865659 : Hello
2021-03-30 20:54:19.868115 : Dude


In [17]:
## Similar problem

import json

def decode(data, default={}) :
    try :
        return json.loads(data)
    except :
        return default
    
x = decode("bad data")
x['yo'] = 2
y = decode("more bad")
y['foo'] = 3

print (x, y)

{'yo': 2, 'foo': 3} {'yo': 2, 'foo': 3}


In [20]:
from typing import Optional

def log_typed(message: str, when: Optional[datetime] = None) -> None :
    if  when is None :
        when = datetime.now()
    print (f"{when} : {message}")
    
d = datetime.now()
log("Dude")
sleep(1)
log("Yo")
log("Time machine")

2021-03-30 20:58:47.374178 : Dude
2021-03-30 20:58:48.376118 : Yo
2021-03-30 20:58:48.376431 : Time machine


*Item 25 : Enforce clarity with keyword only and positional only arguments*

In [24]:
## Python 3.8
def safe_division(numerator, denominator, ## /, 
                    *, ignore_overflow=False, ignore_zero_division=False) :
    try :
        return numerator/denominator
    except OverflowError :
        if ignore_overflow : return 0
        raise
    except ZeroDivisionError :
        if ignore_zero_division : return float('inf')
        raise

print (safe_division(5, 0, ignore_zero_division=True))

inf


*Item 26 : Define function decorators with functools.wraps*

In [26]:
def trace(func) :
    def wrapper(*args, **kwargs) :
        result = func(*args, **kwargs)
        print (f"{func.__name__}({args!r}, {kwargs!r}) -> {result!r}")
        return result
    return wrapper
    
@trace
def fib(n) :
    if n < 2 : return 1
    return (fib(n-1) + fib(n-2))

fib(5)

fib((1,), {}) -> 1
fib((0,), {}) -> 1
fib((2,), {}) -> 2
fib((1,), {}) -> 1
fib((3,), {}) -> 3
fib((1,), {}) -> 1
fib((0,), {}) -> 1
fib((2,), {}) -> 2
fib((4,), {}) -> 5
fib((1,), {}) -> 1
fib((0,), {}) -> 1
fib((2,), {}) -> 2
fib((1,), {}) -> 1
fib((3,), {}) -> 3
fib((5,), {}) -> 8


8

In [28]:
print (fib)

<function trace.<locals>.wrapper at 0x7fed3a0c9290>


In [30]:
from functools import wraps

def trace(func) :
    @wraps(func)
    def wrapper(*args, **kwargs) :
        result = func(*args, **kwargs)
        print (f"{func.__name__}({args!r}, {kwargs!r}) -> {result!r}")
        return result
    return wrapper

@trace
def fib(n) :
    if (n < 2) : return 1
    return fib(n-1) + fib(n-2)

fib(5)

fib((1,), {}) -> 1
fib((0,), {}) -> 1
fib((2,), {}) -> 2
fib((1,), {}) -> 1
fib((3,), {}) -> 3
fib((1,), {}) -> 1
fib((0,), {}) -> 1
fib((2,), {}) -> 2
fib((4,), {}) -> 5
fib((1,), {}) -> 1
fib((0,), {}) -> 1
fib((2,), {}) -> 2
fib((1,), {}) -> 1
fib((3,), {}) -> 3
fib((5,), {}) -> 8


8

In [31]:
print (fib)

<function fib at 0x7fed3a0c9710>
